In [249]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

In [250]:
# importing csv and adding new columbs

mushroom_df = pd.read_csv("mushroom_data.csv")

column_headers = ['Edibility', 'Cap Shape', 'Cap Surface', 'Cap Color', 'Bruises', 'Odor', 'Gill Attachment', 'Gill Spacing',
                  'Gill Size', 'Gill Color', 'Stalk Shape', 'Stalk Root', 'Stalk Surface Above Ring', 'Stalk Surface Below Ring',
                  'Stalk Color Above Ring', 'Stalk Color Below Ring', 'Veil Type', 'Veil Color', 'Ring Number', 'Ring Type',
                  'Spore Print Color', 'Population', 'Habitat']

mushroom_df = pd.read_csv("mushroom_data.csv", names=column_headers, header=None)

mushroom_df.dropna(inplace=True)

mushroom_df



# Cap Shape = bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
# Cap Surface = ibrous=f,grooves=g,scaly=y,smooth=s
# Cap Color = brown=n,buff=b,cinnamon=c,gray=g,green=r, pink=p,purple=u,red=e,white=w,yellow=y
# Odor = almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s

print(unique_vals)
mushroom_df

['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']


,Edibility,Cap Shape,Cap Surface,Cap Color,Bruises,Odor,Gill Attachment,Gill Spacing,Gill Size,Gill Color,...,Stalk Surface Below Ring,Stalk Color Above Ring,Stalk Color Below Ring,Veil Type,Veil Color,Ring Number,Ring Type,Spore Print Color,Population,Habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [251]:
# makes the letters into numbers
label_encoder = LabelEncoder()
mushrooms_df_encoded = mushrooms_df.apply(label_encoder.fit_transform)

X = mushrooms_df_encoded[['Cap Shape', 'Cap Surface', 'Cap Color', "Odor"]]
y = mushrooms_df_encoded['Edibility']
X_mush_train, X_mush_test, y_mush_train, y_mush_test = train_test_split(X, y, test_size=0.2, random_state=42)


# make dummy variables w onehot
categorical_columns = ['Cap Shape', 'Cap Surface', 'Cap Color', 'Odor']
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
mush_preprocessor = make_column_transformer((categorical_transformer, categorical_columns),)


X_train_preprocessed = mush_preprocessor.fit_transform(X_mush_train)
X_test_preprocessed = mush_preprocessor.transform(X_mush_test)

#### KNeighbors
knn_spec = KNeighborsClassifier()
classifier.fit(X_train_preprocessed, y_train)

mush_fit = make_pipeline(mush_preprocessor, KNeighborsClassifier()).fit(X_train, y_train)
mush_fit


mush_preprocessor
mush_fit

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Cap Shape', 'Cap Surface',
                                                   'Cap Color', 'Odor'])])),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [252]:
# new predicted column
mush_test_predictions = mush_fit.predict(X_test[['Cap Shape', 'Cap Surface', 'Cap Color', 'Odor']])
mush_test_predictions = pd.concat(
    [
        X_test.reset_index(drop=True), y,
        pd.DataFrame(mush_test_predictions, columns=["Predicted"]),
    ],
    axis=1,
)

mush_test_predictions.head(20)


,Cap Shape,Cap Surface,Cap Color,Odor,Edibility,Predicted
0,2.0,0.0,4.0,5.0,1,0
1,2.0,2.0,2.0,8.0,0,1
2,5.0,3.0,4.0,2.0,0,1
3,2.0,3.0,3.0,5.0,1,0
4,2.0,2.0,2.0,7.0,0,1
5,5.0,3.0,4.0,8.0,0,1
6,5.0,2.0,3.0,2.0,0,1
7,5.0,3.0,9.0,2.0,0,1
8,2.0,0.0,4.0,5.0,1,0
9,2.0,3.0,4.0,5.0,0,0


In [244]:
# accuracy percentage
X_acc_test = mush_test_predictions[['Cap Shape', 'Cap Surface', 'Cap Color', 'Odor']]

y_acc_test = mush_test_predictions["Edibility"]

mush_prediction_accuracy = mush_fit.score(X_acc_test, y_acc_test)
mush_prediction_accuracy

0.5626538650910882